In [1]:
#!conda update anaconda-navigator  
#!conda update navigator-updater
#!pip install --upgrade pip

In [26]:
!pip install imutils
!pip3 install opencv-python
!pip3 install playsound
!pip3 install cmake
#cmake configuration and visual_studio sdk needed to local pip of dlib
!pip3 install dlib
!pip3 install --upgrade google-api-python-client
!pip3 install google-cloud
!pip3 install google-cloud-vision
!pip install imageio-ffmpeg

In [18]:
#import libraries

#for computind euclidean distance (EAR) between eye's points
from scipy.spatial import distance as dist
#basic image processing functions such as translation, rotation, resizing
from imutils.video import VideoStream
from imutils import face_utils
#to play our alarm in a separate thread from the main thread to ensure that the heavy computation doesn’t pause execution while the alarm sounds.
from threading import Thread
import numpy as np
#to actually play our WAV/MP3 alarm
from IPython.display import Audio
from IPython.display import display
import argparse
import imutils
import time
#to detect and localize facial landmarks
import dlib
import cv2
#for file upload from local device
#from google.colab import files

#for data compression
import bz2
#video_editing
from PIL import Image
import imageio
import io


In [4]:
#download pre-trained face_landmark predictor
#wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

In [5]:
#load alarm sound from local device to drive 
#uploaded = files.upload()

#play sound fun
def play_alarm(file_path):
    wn = Audio(path, autoplay=True) ##
    display(wn)

#test fun
path = 'alarm.mp3'
play_alarm(path)

In [6]:
#Compute EAR index
def compute_EAR(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear

In [7]:
#retrieve pre-trained shape predictor with 68 face landmarks
"""has to be done just once"""
"""
uncompressed_folder = bz2.BZ2File('shape_predictor_68_face_landmarks.dat.bz2').read()
f = open('./shape_predictor_68_face_landmarks.dat', 'wb+',)
f.write(uncompressed_folder)
f.close()
"""

"\nuncompressed_folder = bz2.BZ2File('shape_predictor_68_face_landmarks.dat.bz2').read()\nf = open('./shape_predictor_68_face_landmarks.dat', 'wb+',)\nf.write(uncompressed_folder)\nf.close()\n"

In [8]:
#retrieve default face detector
detector = dlib.get_frontal_face_detector()
#retrieve default predictor
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")

In [9]:
#indexes for eye landamarks
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [10]:
#loop and process frames from the video stream
#frame acquisition should be in a dedicated thread
#a thread 1 should be dedicated to RT frames acquisition
#main thread should be dedicated to process frames once available

class VideoStreamWidget(object):
    def __init__(self, src=0):
        self.number_of_closed_frames = 0
        self.number_of_opened_frames = 0
        self.capture = cv2.VideoCapture(src)
        # Start the thread to read frames from the video stream
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True
        self.thread.start()

    def update(self):
        # Read the next frame from the stream in a different thread
        while True:
            if self.capture.isOpened():
                (self.status, self.frame) = self.capture.read()
            time.sleep(.01)

    def get_frame(self):
        # Display frames in main program
        #cv2.imshow('frame', self.frame)
        
        key = cv2.waitKey(0)
        if key == ord('q'):
            self.capture.release()
            cv2.destroyAllWindows()
            exit(1)
            
        return self.frame



In [39]:
#wrap the image with facial landmark
def edit_image(frame,r_eye,l_eye):
    leftEyeHull = cv2.convexHull(l_eye)
    rightEyeHull = cv2.convexHull(r_eye)
    cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
    cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    #Calcolo PERCLOS (occorre ridefinire finestra mobile e soglia dinamica per l'EAR)
    
def PERCLOS_f(drawsiness_videodetector):
    if(ear < 0.3):
        drawsiness_videodetector.number_of_closed_frames = drawsiness_videodetector.number_of_closed_frames + 1
    else:
        drawsiness_videodetector.number_of_opened_frames = drawsiness_videodetector.number_of_opened_frames + 1

    perclos = (number_of_closed_frames / (number_of_closed_frames+number_of_opened_frames) )*100
    cv2.putText(frame, "PERCLOS: {:.2f}".format(perclos), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    color = (0, 255, 0)
    if(perclos <= 3.75):
        color = (0, 255, 0)
    elif (perclos <=7.5):
        color = (0,200,0)
    elif (perclos <= 11.25):
        color = (222,255,0)
    elif (perclos <= 15 ):
        color = (255,196,0)
    else:
        color = (255,0,0)
        path = 'alarm.wav'
        play_alarm(path)

    cv2.rectangle(frame, (2,2), (448, 235), color, 4)
    plt.imshow(frame, interpolation='nearest')
    plt.show()

"""
def edit_video():
    with imageio.get_writer('/path/to/movie.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
"""

"\ndef edit_video():\n    with imageio.get_writer('/path/to/movie.gif', mode='I') as writer:\n        for filename in filenames:\n            image = imageio.imread(filename)\n            writer.append_data(image)\n"

In [40]:
#activation if an indipendent thread to manage frames acquisition
drawsiness_videodetector = VideoStreamWidget()  

In [41]:
"""il codice serve per editare i frame istante per istante e produrre una sorgente di output editata"""


#main program
while True:
        try:
            #retrieve last frame from second thread
            new_frame_img = video_stream_widget.get_frame()

            #conversion to numpy array
            frame = np.array(new_frame_img)
            """
            byteImgIO = io.BytesIO()
            byteImg = Image.open(new_frame_img)
            byteImg.save(byteImgIO, "PNG")
            byteImgIO.seek(0)
            byteImg = byteImgIO.read()
            dataBytesIO = io.BytesIO(new_frame_img)
            frame = np.array(Image.open(dataBytesIO))
            """

            #img PREPROCESSING
            res_frame = imutils.resize(frame, width=450)
            gray_res_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            #face detection
            face_frame = detector(gray_res_frame, 0)

            #landmark prediction and texture
            for pixel in face_frame:

                landmarks_idx = predictor(gray_res_frame, pixel)
                landmarks_idx = face_utils.shape_to_np(landmarks_idx)

                #extract eye-index ### avoid once custum model is fitted 
                leftEye = landmarks_idx[lStart:lEnd]
                rightEye = landmarks_idx[rStart:rEnd]
                leftEAR = compute_EAR(leftEye)
                rightEAR = compute_EAR(rightEye)

                #avg or take the min ???
                ear = (leftEAR + rightEAR) / 2.0

                #composing final image
                edit_image(frame,rightEye,leftEye)
                #display final image
                fimg = Image.fromarray(np.uint8(res_frame))
                #PERCL ESTIMATION AND ALERT LEVEL DETECTION
                
                #plot img in a videostream fashion
                cv2.rectangle(frame, (2,2), (448, 235), color, 4)
                plt.imshow(frame, interpolation='nearest')
                plt.show()

        except AttributeError:
            pass


ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
"""BLINK FILTERING:: to develop"""

In [ ]:
#Calcolo PERCLOS (occorre ridefinire finestra mobile e soglia dinamica per l'EAR)
def PERCLOS_f(drawsiness_videodetector):
if(ear < 0.3):
    drawsiness_videodetector.number_of_closed_frames = drawsiness_videodetector.number_of_closed_frames + 1
else:
    drawsiness_videodetector.number_of_opened_frames = drawsiness_videodetector.number_of_opened_frames + 1

perclos = (number_of_closed_frames / (number_of_closed_frames+number_of_opened_frames) )*100
cv2.putText(frame, "PERCLOS: {:.2f}".format(perclos), (10, 30),
cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
color = (0, 255, 0)
if(perclos <= 3.75):
    color = (0, 255, 0)
elif (perclos <=7.5):
    color = (0,200,0)
elif (perclos <= 11.25):
    color = (222,255,0)
elif (perclos <= 15 ):
    color = (255,196,0)
else:
    color = (255,0,0)
    path = 'alarm.wav'
    play_alarm(path)

cv2.rectangle(frame, (2,2), (448, 235), color, 4)
plt.imshow(frame, interpolation='nearest')
plt.show()